In [ ]:
import os
import sys
import urllib.request
from tqdm import tqdm

import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.chrome.options import Options

import time
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 지역 이름만 계속 바꿔주면 됩니다.
area = 'Gangnam'

#csv_file = "rest_concat"
client_id = "789Xk04GARJpb4omVvUq" # 개발자센터에서 발급받은 Client ID 값
client_secret = "oynUXBN1cW" # 개발자센터에서 발급받은 Client Secret 값

In [ ]:
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

data = pd.read_csv(f'./area_csv/{area}/rest_concat.csv')
#rest_list = list(data['restaurant'].values)

data['x'] = 0
data['y'] = 0
data['image'] = 'tem'

In [ ]:
for idx in tqdm(range(len(data))): # range(3)
    # 이미지 크롤링
    driver = webdriver.Chrome(executable_path='./chromedriver', options=chrome_options)
    url = data['url'][idx]
    driver.get(f"https://m.place.naver.com/restaurant/{url}/home")
    name_list = driver.find_elements(By.ID, "og:image")
    if len(name_list) > 0:
        data['image'][idx] = name_list[0].get_attribute('content')

    # x,y 크롤링
    encText = urllib.parse.quote(data['restaurant'][idx])
    url = "https://openapi.naver.com/v1/search/local?query=" + encText # JSON 결과
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    time.sleep(0.11)
    if(rescode==200):
        response_body = response.read()
        data['x'][idx] =  response_body.decode('utf-8').split("\"")[-6]
        data['y'][idx] =  response_body.decode('utf-8').split("\"")[-2]
    else:
        print("Error Code:" + rescode)
    

In [ ]:
data.to_csv(f'./area_csv/{area}/rest_concat_{area}.csv', index=False)